In [14]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.externals import joblib

import lightgbm as lgb
from lightgbm import LGBMClassifier

from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from sklearn.metrics import log_loss, make_scorer
from functools import partial

from sklearn.model_selection import train_test_split

import gc
import time
import warnings
warnings.filterwarnings('ignore')

In [15]:
train = pd.read_csv('../dataset/train_20200226.csv', index_col = 0)
test = pd.read_csv('../dataset/test_20200226.csv', index_col = 0)
sample_submission = pd.read_csv('../dataset/sample_submission.csv', index_col = 0)

In [16]:
column_number = {}

for i, column in enumerate(sample_submission.columns):
    column_number[column] = i

def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [17]:
#bins= [0, 100, 200, 300, 400, 500, 600, 700, 800, 900 ,1000]
#labels = ['0', '1','2','3','4','5','6','7','8','9']

#train['fiberID2'] = pd.cut(train['fiberID'], bins= bins, labels = labels)
#test['fiberID2'] = pd.cut(test['fiberID'], bins= bins, labels = labels)

In [18]:
train_x = train.drop(columns = ['type', 'type_num'], axis = 1)
train_y = train['type_num']

test_x = test

In [19]:
train_x.drop(columns = 'fiberID', inplace = True)
test_x.drop(columns = 'fiberID', inplace = True)

In [20]:
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe_train = pd.DataFrame(ohe.fit_transform(train_x[['fiberID2']]).toarray(),
                         columns = ['fiber_0', 'fiber_1', 'fiber_2', 'fiber_3', 'fiber_4', 'fiber_5', 'fiber_6', 'fiber_7'])
ohe_test = pd.DataFrame(ohe.fit_transform(test_x[['fiberID2']]).toarray(),
                        columns = ['fiber_0', 'fiber_1', 'fiber_2', 'fiber_3', 'fiber_4', 'fiber_5', 'fiber_6', 'fiber_7'])
ohe_train.head()

,fiber_0,fiber_1,fiber_2,fiber_3,fiber_4,fiber_5,fiber_6,fiber_7
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [21]:
idx_list = [x for x in range(199991,210000)]
ohe_test['id'] = idx_list
ohe_test.set_index('id', inplace = True)

In [22]:
train_x = pd.concat([train_x, ohe_train], axis = 1)
test_x = pd.concat([test_x, ohe_test], axis = 1)
train_x.drop('fiberID2', axis = 1, inplace = True)
test_x.drop('fiberID2', axis = 1, inplace = True)
train_x.head()

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,...,star_spectrum_model__M,star_spectrum_model__O,fiber_0,fiber_1,fiber_2,fiber_3,fiber_4,fiber_5,fiber_6,fiber_7
0,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,...,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,...,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
#train_x.drop(columns = ['psf_cluster', 'fiber_cluster', 'model_cluster'], inplace = True)
#test_x.drop(columns = ['psf_cluster', 'fiber_cluster', 'model_cluster'], inplace = True)

# Modeling

In [11]:
space = {'objective' : 'multiclass',
         'num_class' : 19,
              'metric' : 'multi_logloss',
              'boosting' : 'gbdt',
    'max_depth':  hp.choice('max_depth', list(range(4, 10, 2))),
    'reg_alpha' : hp.uniform('reg_alpha', 0.01, 0.4),
    'reg_lambda': hp.uniform('reg_lambda', 0.01, 0.4),
    'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.3, 0.9),
    'gamma' : hp.uniform('gamma', 0.01, .7),
    'num_leaves' : hp.choice('num_leaves', list(range(20,150,10))),
    'min_child_samples': hp.choice('min_child_samples', list(range(100,150,10))),
    'subsample': hp.choice('subsample', [.2, .4, .5, .6, .7, .8, .9]),
    'feature_fraction' : hp.uniform('feature_fraction', .4, .8),
    'bagging_fraction' : hp.uniform('bagging_fraction', .5, .9)
}

In [12]:
def evaluate_metric(params):
    time1 = time.time()
    print("******************** New Run *********************")
    print('Params : {}'.format(params))
    FOLDS = 5
    count = 1
    
    skf = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = 42)
    y_preds = np.zeros(test.shape[0])
    score_mean = 0
    
    for tr_idx, val_idx in skf.split(train_x, train_y):
        X_tr, X_vl = train_x.iloc[tr_idx, :], train_x.iloc[val_idx, :]
        y_tr, y_vl = train_y.iloc[tr_idx], train_y.iloc[val_idx]
        
        lgbtrain = lgb.Dataset(X_tr, label = y_tr)
        lgbval = lgb.Dataset(X_vl, label = y_vl)
        
        lgb_clf = lgb.train(params, lgbtrain, 1250, valid_sets = [lgbval],
                           categorical_feature = 'auto',
                           verbose_eval = 0,
                           early_stopping_rounds = 50)
        
        lgb_pred = lgb_clf.predict(X_vl, num_iteration = lgb_clf.best_iteration)
        
        score = make_scorer(log_loss)(lgb_clf, X_vl, y_vl)
        
        score_mean += score
        print('{} CV - Score: {}'.format(count, round(score,4)))
        
        count += 1
    
    time2 = time.time() - time1
    print('Total Time Run : {}'.format(round(time2/60, 2)))
    gc.collect()
    print('Mean Log Loss : {}'.format(score_mean/FOLDS))
    del X_tr, X_vl, y_tr, y_vl, lgb_clf, score
    
    return (score_mean/FOLDS)

In [ ]:
best = fmin(fn = evaluate_metric,
            space = space,
            algo = tpe.suggest,
            max_evals = 10)

best_params = space_eval(space,best)
print("Best HyperParameters :", best_params)

******************** New Run *********************                                                                     
Params : {'bagging_fraction': 0.5801083010711567, 'boosting': 'gbdt', 'colsample_bytree': 0.8689531437411466, 'feature_fraction': 0.7602261581764034, 'gamma': 0.5871936066413894, 'learning_rate': 0.13387612512136307, 'max_depth': 8, 'metric': 'multi_logloss', 'min_child_samples': 120, 'num_class': 19, 'num_leaves': 60, 'objective': 'multiclass', 'reg_alpha': 0.09129547636263352, 'reg_lambda': 0.1610169402817791, 'subsample': 0.5}
1 CV - Score: 0.373                                                                                                    
2 CV - Score: 0.3802                                                                                                   
3 CV - Score: 0.3843                                                                                                   
4 CV - Score: 0.3529                                                                  

Best HyperParameters : {'bagging_fraction': 0.4778711498115298, 'boosting': 'gbdt', 'colsample_bytree': 0.530825962221535, 'feature_fraction': 0.6685108367216006, 'gamma': 0.3089418643344476, 'learning_rate': 0.037576672680065454, 'max_depth': 8, 'metric': 'multi_logloss', 'min_child_samples': 100, 'num_class': 19, 'num_leaves': 140, 'objective': 'multiclass', 'reg_alpha': 0.15641799752802943, 'reg_lambda': 0.31890794153928625, 'subsample': 0.7} (0225 Dataset)


Params : {'bagging_fraction': 0.5131338905319136, 'boosting': 'gbdt', 'colsample_bytree': 0.32712831802310366, 'feature_fraction': 0.6516096245942946, 'gamma': 0.22556698033265804, 'learning_rate': 0.052665252375510826, 'max_depth': 4, 'metric': 'multi_logloss', 'min_child_samples': 130, 'num_class': 19, 'num_leaves': 70, 'objective': 'multiclass', 'reg_alpha': 0.36137884251729446, 'reg_lambda': 0.20533863661000978, 'subsample': 0.5}
(0226 Dataset)

In [27]:
best_params = {'bagging_fraction': 0.5131338905319136, 'boosting': 'gbdt', 'colsample_bytree': 0.32712831802310366, 'feature_fraction': 0.6516096245942946, 'gamma': 0.22556698033265804, 'learning_rate': 0.052665252375510826, 'max_depth': 8, 'metric': 'multi_logloss', 'min_child_samples': 130, 'num_class': 19, 'num_leaves': 140, 'objective': 'multiclass', 'reg_alpha': 0.36137884251729446, 'reg_lambda': 0.20533863661000978, 'subsample': 0.5}

In [28]:
X_tr, X_vl, y_tr, y_vl = train_test_split(train_x, train_y, random_state = 42, shuffle = True)

X_test = test_x

lgb_train = lgb.Dataset(X_tr, label = y_tr)
lgb_valid = lgb.Dataset(X_vl, label = y_vl)

lgb_model = lgb.train(best_params,
                      lgb_train, 1250,
                      valid_sets = [lgb_train, lgb_valid],
                      verbose_eval = 10,
                      categorical_feature = 'auto',
                      early_stopping_rounds = 50)

Training until validation scores don't improve for 50 rounds
[10]	training's multi_logloss: 1.21392	valid_1's multi_logloss: 1.2325
[20]	training's multi_logloss: 0.838716	valid_1's multi_logloss: 0.864306
[30]	training's multi_logloss: 0.635906	valid_1's multi_logloss: 0.666804
[40]	training's multi_logloss: 0.517379	valid_1's multi_logloss: 0.553032
[50]	training's multi_logloss: 0.445027	valid_1's multi_logloss: 0.484741
[60]	training's multi_logloss: 0.398356	valid_1's multi_logloss: 0.441882
[70]	training's multi_logloss: 0.367324	valid_1's multi_logloss: 0.414538
[80]	training's multi_logloss: 0.34546	valid_1's multi_logloss: 0.396291
[90]	training's multi_logloss: 0.329398	valid_1's multi_logloss: 0.383905
[100]	training's multi_logloss: 0.316983	valid_1's multi_logloss: 0.375099
[110]	training's multi_logloss: 0.306837	valid_1's multi_logloss: 0.368683
[120]	training's multi_logloss: 0.298405	valid_1's multi_logloss: 0.363991
[130]	training's multi_logloss: 0.291382	valid_1's m

In [29]:
import time
time = time.gmtime(time.time()) 
year = time[0]
month = time[1]
day = time[2]

joblib.dump(lgb_model, '../model/LGB_OPT_{}{}{}_3472_20020226.pkl'.format(year, month, day))

['../model/LGB_OPT_2020226_3472_20020226.pkl']

In [30]:
pred = lgb_model.predict(test_x, num_iteration = lgb_model.best_iteration)

In [31]:
submission = pd.DataFrame(data = pred, columns = sample_submission.columns, index = sample_submission.index)

In [32]:
submission.to_csv('../submission/lgb_opt_3472.csv')